In [1]:
from sagas.ofbiz.services import OfService as s, oc, track
from sagas.ofbiz.entities import OfEntity as e

oc.import_package('org.apache.ofbiz.service.ServiceDispatcher')

2021-04-09 18:30:32,681 - sagas.ofbiz.connector - INFO - connect to py4j-gateway localhost 22333


In [2]:
oc.j.ServiceDispatcher.LRU_LOG_SIZE

200

In [3]:
oc.import_package('com.adapters.objects.RequestData')
oc.import_package('com.adapters.objects.ResponseData')
request = oc.j.RequestData()
response = oc.j.ResponseData()

In [4]:
security=oc.dispatcher.getSecurity()
request.setAttribute("security", security)
request.setAttribute("delegator", oc.delegator)
request.setAttribute("dispatcher", oc.dispatcher)
session = request.getSession()
session.setAttribute("orderMode", None)

In [5]:
oc.import_package('org.apache.ofbiz.order.shoppingcart.ShoppingCartEvents')
result = oc.j.ShoppingCartEvents.routeOrderEntry(request, response)
result

'init'

In [6]:
user_login=oc.gateway.entry_point.getUserLogin()
user_login.get('userLoginId')

'system'

In [7]:
request.setParameter("orderMode", "SALES_ORDER")
request.setParameter("productStoreId", "9000")
request.setParameter("partyId", "DemoCustomer")
request.setParameter("currencyUom", "USD")
session.setAttribute("userLogin", user_login)

In [8]:
result = oc.j.ShoppingCartEvents.initializeOrderEntry(request, response)
result

'success'

In [9]:
result = oc.j.ShoppingCartEvents.setOrderCurrencyAgreementShipDates(request, response)
result

'success'

In [10]:
product_id= "GZ-1000"
request.setParameter("add_product_id", product_id)
result = oc.j.ShoppingCartEvents.addToCart(request, response)
result

'success'

In [11]:
oc.import_package('org.apache.ofbiz.order.shoppingcart.CheckOutEvents')
CheckOutEvents=oc.j.CheckOutEvents

In [12]:
request.setParameter("checkoutpage", "quick")
request.setParameter("shipping_contact_mech_id", "9015")
request.setParameter("shipping_method", "GROUND@UPS")
request.setParameter("checkOutPaymentId", "EXT_COD")
request.setParameter("is_gift", "false")
request.setParameter("may_split", "false")
request.setAttribute("shoppingCart", None)

result = CheckOutEvents.setQuickCheckOutOptions(request, response)
result

'success'

In [13]:
result = CheckOutEvents.createOrder(request, response)
result

'sales_order'

In [14]:
result = CheckOutEvents.processPayment(request, response)
result

'success'

In [15]:
oc.dispatcher.runAsync("sendOrderConfirmation", None)

In [16]:
result = oc.j.ShoppingCartEvents.destroyCart(request, response)
result

'success'

In [17]:
oc.import_package('org.apache.ofbiz.entity.util.EntityQuery')
q=oc.j.EntityQuery.use(oc.delegator)

In [18]:
import py4j

In [19]:
select_from=py4j.java_gateway.get_method(q, 'from')
select_from('Person')

JavaObject id=o11

In [20]:
query=select_from("OrderHeader")
query.getClass().getName()

'org.apache.ofbiz.entity.util.EntityQuery'

In [21]:
select_where=py4j.java_gateway.get_method(query, 'where')

In [22]:
# query_o=select_where("orderTypeId", "SALES_ORDER")

In [23]:
# order_header=select_from("OrderHeader").where("orderTypeId", "SALES_ORDER").orderBy("-entryDate").queryFirst()

In [25]:
oc.import_package('com.adapters.srv.ECommerceHelper')
helper=oc.j.ECommerceHelper(oc.dispatcher, oc.delegator)

In [28]:
order_header=helper.getLatestSalesOrder()
order_header.get('orderId')

'WSCO10010'

In [37]:
order_input=oc.jmap()
order_input.put('orderId', order_header.get('orderId'))
order_input.put('invoicePerShipment', 'Y')
order_input.put('userLogin', user_login)
service_result = oc.dispatcher.runSync("updateOrderHeader", order_input)

In [38]:
service_result

{'responseMessage': 'success'}

In [39]:
print(service_result.get('errorMessageList'))

None
